In [1]:
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from gensim import corpora, models
import pyLDAvis.gensim as gensimvis
import pyLDAvis

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#stops = set(stopwords.words("english"))

In [93]:
f=open("Data/comments.txt",'r')
docSet = f.readlines()
f.close()

In [127]:
added_stopwords=["car","cars", "nissan", "altima", "sentra", "sentras", "vehicle", "vehicles", "get", \
            "really", "new", "rogue", "rouge", "pathfinder", "maxima", "versa", "murano", "us",\
            "yet", "honda", "etc", "xterra", "suv", "quest", "one", "two", "toyota","black", \
            "although", "lexus", "infiniti", "also", "would", "fx", "want", "month", "mom", "wife","father"\
            "much","wish","use","girl","boy","damn","want","sv","kids","nisssan","nissans","day","went","given", \
            "sl","monday","infinity","of","for"]


In [138]:
replaceList=[["-",""],["don\'t know", "dont_know"],["responce","response"],["great","good"],["a bit",""],\
             ["a/c","air_conditioning"],["n/a","not applicable"],["at this time",""],["mpg","fuel economy"],\
             ["gas mileage","fuel economy"],["love","like"],["so far",""],[" ac ", "air conditioning"],["sales staff","dealership"], \
             ["fuel efficiency","fuel economy"],["i.e.","ie"],["ie","for example"],["overall quality","overall_quality"]]

In [139]:
# Create p_stemmer of class PorterStemmer
#p_stemmer = PorterStemmer()
SBstemmer = SnowballStemmer("english")
lemmatizer=WordNetLemmatizer()
#create a tokenizer
tokenizer = RegexpTokenizer(r'\w+')
#create a list of stop words
stops=set(stopwords.words('english')+added_stopwords)

token_texts=[]
non_english=0
for i in docSet:
    #if len(i)<350:
        # clean and tokenize document string
    raw = i.lower()
        #remove non-letter chars
    for j in replaceList:
        raw=raw.replace(j[0],j[1])
    letters_only = re.sub("[^a-zA-Z]", " ", raw)
        #tokenize words
    tokens = tokenizer.tokenize(letters_only)
    token_texts.append(tokens)

bigrams=models.phrases.Phrases(token_texts)
trigrams=models.phrases.Phrases(bigrams[token_texts])
bi_texts=trigrams[bigrams[token_texts]]
texts=[]
for i in bi_texts:
    # remove stop words from tokens and lemmatize
    #stopped_tokens = [lemmatizer.lemmatize(j) for j in i if not j in stops]
    stopped_tokens = [SBstemmer.stem(j) for j in i if not j in stops]
    
    # add tokenized document to list
    texts.append(stopped_tokens)

In [126]:
stops=set(stopwords.words('english')+added_stopwords)
count=0
word="ou"
for text in texts:
    if word in text:
        break

print(text)
print(texts.index(text))

print(docSet[texts.index(text)])
print(lemmatizer.lemmatize("out"))

[u'owners_manual', u'confusing', u'organized', u'well', u'index', u'terrible', u'does_not', u'list', u'subsystem', u'e', u'all_wheel_drive', u'subsystem', u'listed', u'index', u'go', u'several', u'plair_conditioninges', u'figure', u'ou', u'work', u'please_fix', u'previous', u'issue']
141801
The owners manual is very confusing to use. It is not organized very well. The index is terrible and does not list the subsystems of the car. I.E. The all wheel drive subsystem is not listed in the index. You have to go to several places to figure ou how it works.  Please fix the previous issues.

out


In [140]:
# turn our tokenized documents into a id <-> term dictionary

dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=.08, no_above=.5, keep_n=None)
dictionary.compactify()

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [141]:
# generate LDA model
ldamodel = models.ldamodel.LdaModel(corpus, id2word = dictionary, alpha='auto', num_topics=100, passes=3)

In [75]:
x=100
print(texts[x])
ldaVec=ldamodel[corpus[x]]
print(docSet[x])
print(ldaVec)

ldamodel.print_topic(30)
#ldamodel.print_topic(106)
#ldamodel.print_topic(135)

[u'like', u'everyth']
At this time I like everything.

[(13, 0.27431741609290611), (18, 0.010087539652073416), (30, 0.24371244535374029), (46, 0.014067475673862219), (66, 0.010063693066640971)]


u'0.581*everyth + 0.126*navigation_system + 0.068*good + 0.036*reput + 0.029*exterior_look + 0.027*direct + 0.016*have_owned_nissan + 0.014*year_old + 0.012*immedi + 0.011*agre'

In [ ]:
print(texts[3574])

In [142]:
vis_data = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis_data)

In [ ]:
ldacorpus=ldamodel[corpus]

In [51]:
#print(dictionary)
freq=[]
for i in xrange(27586):
    freq.append(sum(dict(doc).get(i, 0) for doc in corpus))

KeyboardInterrupt: 